In [2]:
import numpy as np
import pandas as pd

import tensorflow
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical

In [7]:
''' defining some useful funcion '''

def loading_data(train=40000, test=10000, full=False):
    ''' reading data '''
    if full:
        s_df = pd.read_csv('sudoku.csv').values
    else:
        ''' converting s_df into array '''
        s_df = next(
            pd.read_csv('sudoku.csv', chunksize=(train + test))).values
        
    ''' transposing data '''
    q, sols = s_df.T
    X = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in q])
    y = np.array([np.reshape([int(d) for d in flatten_grid], (9, 9)) for flatten_grid in sols])
    
    return (X[:train], y[:train]), (X[train:], y[train:])


def diff(grids_true, grids_pred):
    """ This function shows how well predicted quizzes fit to actual solutions. """
    return (grids_true != grids_pred).sum((1, 2))


def del_digits(X, delete=1):
    """ This function is used to create sudoku quizzes from solutions """
    grids = X.argmax(3)  # get the grid in a (9, 9) integer shape
    for grid in grids:
        grid.flat[np.random.randint(0, 81, delete)] = 0  # generate blanks (replace = True)
        
    return to_categorical(grids)


def batch_smart_solve(grids, solver):
    """ This function is ugly, feel free to optimize the code """
    grids = grids.copy()
    for _ in range((grids == 0).sum((1, 2)).max()):
        pred = np.array(solver.predict(to_categorical(grids)))  # get predictions
        probs = pred.max(2).T  # get highest probability for each 81 digit to predict
        values = pred.argmax(2).T + 1  # get corresponding values
        zeros = (grids == 0).reshape((grids.shape[0], 81))  # get blank positions

        for grid, prob, value, zero in zip(grids, probs, values, zeros):
            '''  don't try to fill already completed grid'''
            if any(zero):  
                '''focus on blanks only'''
                where = np.where(zero)[0]  
                ''' best score FOR A ZERO VALUE (confident blank) '''
                conf_pos = where[prob[zero].argmax()] 
                ''' get corresponding value '''
                conf_val = value[conf_pos]  
                ''' fill digit inplace '''
                grid.flat[conf_pos] = conf_val  
    return grids

In [9]:
inp_shape = (9, 9, 10)
(_, y_train), (X_test, y_test) = loading_data() 

''' one-hot encoding'''
X_train = to_categorical(y_train).astype('float32')
X_test = to_categorical(y_test).astype('float32')

y_train = to_categorical(y_train-1).astype('float32') 
y_test = to_categorical(y_test-1).astype('float32')   

In [15]:
''' Model '''

''' container '''
model = Sequential()

''' 1st Hidden Layer '''
model.add(Dense(64, activation='relu', input_shape=inp_shape))
model.add(Dropout(0.4))

''' 2nd Hidden Layer '''
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))

''' flattening output '''
model.add(Flatten())

''' Input layer '''
inp = Input(shape=inp_shape)  
features = model(inp)  # commons features

''' Classification Layer '''
out = [Dense(9, activation='softmax')(features) for i in range(81)]

''' final model '''
model = Model(inp, out)  

''' compile the model '''
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
''' training '''
model.fit(del_digits(X_train, 0), [y_train[:, i, j, :] for i in range(9) for j in range(9)], batch_size=128, epochs=1, 
           verbose=1)

313/313 [==============================] - 213s 470ms/step - loss: 98.4436 - dense_2_loss: 1.2099 - dense_3_loss: 1.2194 - dense_4_loss: 1.2170 - dense_5_loss: 1.2096 - dense_6_loss: 1.2134 - dense_7_loss: 1.2179 - dense_8_loss: 1.2136 - dense_9_loss: 1.2145 - dense_10_loss: 1.2119 - dense_11_loss: 1.2205 - dense_12_loss: 1.2197 - dense_13_loss: 1.2121 - dense_14_loss: 1.2178 - dense_15_loss: 1.2128 - dense_16_loss: 1.2247 - dense_17_loss: 1.2156 - dense_18_loss: 1.2067 - dense_19_loss: 1.2173 - dense_20_loss: 1.2240 - dense_21_loss: 1.2165 - dense_22_loss: 1.2140 - dense_23_loss: 1.2187 - dense_24_loss: 1.2154 - dense_25_loss: 1.2137 - dense_26_loss: 1.2240 - dense_27_loss: 1.2152 - dense_28_loss: 1.2143 - dense_29_loss: 1.2220 - dense_30_loss: 1.2151 - dense_31_loss: 1.2224 - dense_32_loss: 1.2155 - dense_33_loss: 1.2105 - dense_34_loss: 1.2134 - dense_35_loss: 1.2178 - dense_36_loss: 1.2125 - dense_37_loss: 1.2132 - dense_38_loss: 1.2205 - dense_39_loss: 1.2081 - dense_40_loss: 1.22

In [ ]:
''' Early Stopping '''
es = EarlyStopping(patience=2, verbose=1)

i = 1
for ep, dele in zip([1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], [1, 2, 3, 4, 6, 8, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50, 55]):

    print('Pass n° {} ...'.format(i))
    i += 1
    
    ''' training '''
    model.fit(del_digits(X_train, dele), [y_train[:, i, j, :] for i in range(9) for j in range(9)],
              validation_data=(del_digits(X_train, dele), [y_train[:, i, j, :] for i in range(9) for j in range(9)]), 
              batch_size=128, epochs=ep, verbose=1, callbacks=[es])

Pass n° 1 ...
313/313 [==============================] - 271s 679ms/step - loss: 101.4047 - dense_85_loss: 1.2509 - dense_86_loss: 1.2499 - dense_87_loss: 1.2527 - dense_88_loss: 1.2504 - dense_89_loss: 1.2519 - dense_90_loss: 1.2501 - dense_91_loss: 1.2526 - dense_92_loss: 1.2511 - dense_93_loss: 1.2454 - dense_94_loss: 1.2581 - dense_95_loss: 1.2569 - dense_96_loss: 1.2457 - dense_97_loss: 1.2512 - dense_98_loss: 1.2487 - dense_99_loss: 1.2562 - dense_100_loss: 1.2479 - dense_101_loss: 1.2514 - dense_102_loss: 1.2525 - dense_103_loss: 1.2511 - dense_104_loss: 1.2439 - dense_105_loss: 1.2549 - dense_106_loss: 1.2519 - dense_107_loss: 1.2438 - dense_108_loss: 1.2527 - dense_109_loss: 1.2525 - dense_110_loss: 1.2527 - dense_111_loss: 1.2572 - dense_112_loss: 1.2491 - dense_113_loss: 1.2561 - dense_114_loss: 1.2550 - dense_115_loss: 1.2580 - dense_116_loss: 1.2499 - dense_117_loss: 1.2531 - dense_118_loss: 1.2577 - dense_119_loss: 1.2493 - dense_120_loss: 1.2508 - dense_121_loss: 1.2598 

Pass n° 2 ...
Epoch 1/2
 70/313 [=====>........................] - ETA: 1:59 - loss: 2.2401 - dense_85_loss: 0.0326 - dense_86_loss: 0.0294 - dense_87_loss: 0.0295 - dense_88_loss: 0.0305 - dense_89_loss: 0.0301 - dense_90_loss: 0.0282 - dense_91_loss: 0.0274 - dense_92_loss: 0.0280 - dense_93_loss: 0.0329 - dense_94_loss: 0.0266 - dense_95_loss: 0.0263 - dense_96_loss: 0.0293 - dense_97_loss: 0.0276 - dense_98_loss: 0.0242 - dense_99_loss: 0.0313 - dense_100_loss: 0.0304 - dense_101_loss: 0.0271 - dense_102_loss: 0.0295 - dense_103_loss: 0.0315 - dense_104_loss: 0.0249 - dense_105_loss: 0.0275 - dense_106_loss: 0.0226 - dense_107_loss: 0.0286 - dense_108_loss: 0.0267 - dense_109_loss: 0.0304 - dense_110_loss: 0.0249 - dense_111_loss: 0.0288 - dense_112_loss: 0.0270 - dense_113_loss: 0.0256 - dense_114_loss: 0.0258 - dense_115_loss: 0.0273 - dense_116_loss: 0.0285 - dense_117_loss: 0.0239 - dense_118_loss: 0.0265 - dense_119_loss: 0.0268 - dense_120_loss: 0.0241 - dense_121_loss: 0.029

In [ ]:
'''Evaluating model '''
q = X_test.argmax(3)  
actual = y_test.argmax(3) + 1 

''' make some guesses'''
sg = batch_smart_solve(q, model)   

''' get number of errors on each quizz '''
de = diff(actual, sg)  

'''portion of correct solved quizzes '''
acc = (de == 0).mean()  

In [ ]:
print("""Grid solved:\t {} Correct ones:\t {} Accuracy:\t {}""".format(de.shape[0], (de==0).sum(), acc))